In [25]:
import json
import os
from pathlib import Path

# STEP 0: Open config.json
with open("config.json", "r") as f:
    config = json.load(f)

DATA_DIR = config["data_dir"]
DOWNLOAD_MISSING_DATASET = config["download_missing_dataset"]
GENRES = config["genres"]
INSTALL_MISSING_LIBRARIES = config["install_missing_libraries"]
LIBRARIES = config["libraries"]

script_path = Path("src")
script_download_missing_library = os.path.join(script_path, "download_missing_library.py")

# STEP 1: Check mandatory libraries to run the notebooks
print(f"Find install_missing_libraries = {INSTALL_MISSING_LIBRARIES} in config.json")
if INSTALL_MISSING_LIBRARIES:
    print(f"Missing libraries will be installed automatically")
else:
    print(f"Missing libraries will not be installed automatically")

for pkg in LIBRARIES:
    try:
        __import__(pkg)
    except ImportError:
        if INSTALL_MISSING_LIBRARIES:
            print(f"Installing {pkg}")
            !python "{script_download_missing_library}" {pkg}
        else:
            raise ImportError(f"Mandatory library not found: {pkg}."
                              f"Run the following command to install it: pip install {pkg} or set install_missing_libraries = true in config.json")

# STEP 2: Check the current working directory and validate it with an assertion
# Check OS
SYSTEM_DETECTED = False
if os.name == 'nt':  # Windows
    print("System detect: Windows")
    SYSTEM_DETECTED = True
if os.name == 'posix':  # Linux or Mac
    print("System detect: Linux or Mac")
    SYSTEM_DETECTED = True
assert SYSTEM_DETECTED, "OS not recognized"

# Check pwd directory
print("Actual directory: ", os.getcwd())
assert os.path.basename(os.getcwd()) == "GTZAN_Genre_Recognizer", "The script should be starded from root-project folder with GTZAN_Genre_Recognizer name"

# Set data directory
DATA_DIR = "data\\genres" # default for Windows
if os.name == 'posix':  # default for Linux/Mac
    DATA_DIR = "data/genres" 

#DATASET_EXTENSION = "au"

# STEP 3: Run all the notebooks in sequence
%run notebooks/01_data_setup.ipynb ##TODO: Test download_data.py and invoke it when dataset is not valid (priority very-low)
%run notebooks/02_features.ipynb
#%run 03_training.ipynb
#%run 04_evaluation.ipynb

Find install_missing_libraries = True in config.json
Missing libraries will be installed automatically
System detect: Windows
Actual directory:  c:\Users\rpork\Desktop\Documenti\Studio\MAGISTRALE\Machine Learning\GTZAN_Genre_Recognizer
Check if data exists at the following path:  data\genres
All subfolders found
Number of tracks founds:
blues     : 100
classical : 100
country   : 100
disco     : 100
hiphop    : 100
jazz      : 100
metal     : 100
pop       : 100
reggae    : 100
rock      : 100
Total tracks: 1000


Extracting features:   0%|          | 0/1000 [00:00<?, ?it/s]C:\Users\rpork\AppData\Local\Temp\ipykernel_20772\3929173816.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tempo = 0.0 if np.isnan(tempo) else float(tempo)
Extracting features: 100%|██████████| 1000/1000 [10:47<00:00,  1.55it/s]

Feature matrix shape: (1000, 75)
Labels array shape: (1000,)
Saved data/features/X_features.npy e y_labels.npy
               0          1           2          3          4          5   \
blues -116.626266  76.455132  109.252632  22.034203 -23.163752  19.205610   
blues -111.252853  56.162289  134.152115  27.302845 -24.658051  17.846113   
blues -120.445526  89.953613  117.589798  29.252756 -27.635336  23.429869   
blues -156.545532  73.258926  130.724197  27.111137  -1.480792  20.147448   
blues -127.970406  56.424355  134.440842  28.784885   1.020510  21.237810   

              6          7          8          9   ...          65  \
blues  40.679482  12.692555 -11.888837  13.989555  ...  503.381195   
blues  41.542339  14.267754  -3.075081  11.456511  ...  476.272064   
blues  42.838066  13.932372  -6.757944  17.213970  ...  685.006348   
blues  36.234592  12.286618   0.350913  15.697165  ...  654.350586   
blues  54.439240  12.436433  -0.454471  12.703521  ...  690.172180   

     